In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="123.0.6312.105").install()))


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x7ff6b98ec873 <unknown>
#1 0x7ff6b95e28c6 <unknown>
#2 0x7ff6b9616d34 <unknown>
#3 0x7ff6b9612d3d <unknown>
#4 0x7ff6b965baed <unknown>
#5 0x7ff6b964f343 <unknown>
#6 0x7ff6b9620593 <unknown>
#7 0x7ff6b9620f5e <unknown>
#8 0x7ff6b98b085b <unknown>
#9 0x7ff6b98b47b5 <unknown>
#10 0x7ff6b989e581 <unknown>
#11 0x7ff6b98b5342 <unknown>
#12 0x7ff6b988388f <unknown>
#13 0x7ff6b98db738 <unknown>
#14 0x7ff6b98db90b <unknown>
#15 0x7ff6b98eb9c4 <unknown>
#16 0x7ff6b8b44ac3 <unknown>


In [73]:
import pickle as pk
import requests
from bs4 import BeautifulSoup as BS
from tqdm import tqdm

head = {"User-Agent":
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}
with open("./kbo.pkl","rb") as f:
    abc = pk.load(f)
    kbo = []
    for idx in tqdm(abc):
        url = f"https://www.koreabaseball.com/Futures/Player/HitterDetail.aspx?playerId={idx}"
        bs = BS(requests.get(url,headers=head).text).find('div',{'class':"player_basic"})
        team = BS(requests.get(url,headers=head).text).find('h4',{'id':"h4Team"})
        _key = []
        _values = []
        for i in bs.text.strip().split("\n"):
            a = list(map(lambda x: x.strip(),i.split(":")))
            _key.append(a[0])
            _values.append(a[1])
        b = dict(zip(_key,_values))
        b['팀'] = team.text.strip() 
        kbo.append(b)
    

100%|██████████| 844/844 [03:18<00:00,  4.24it/s]


In [82]:
def get_kbo(player_id):
    url = f"https://www.koreabaseball.com/Futures/Player/HitterDetail.aspx?playerId={player_id}"
    bs= BS(requests.get(url).text).find('div',{'class':'player_basic'})
    team = BS(requests.get(url).text).find('h4',{'id':"h4Team"})
    _keys = []
    _values = []
    for i in bs.text.strip().split("\n"):
        a = list(map(lambda x: x.strip(),i.split(":")))
        _key.append(a[0])
        _values.append(a[1])
    b = dict(zip(_key,_values))
    b['팀'] = team.text.strip()
    return b

In [85]:
import pandas as pd
df = pd.DataFrame([get_kbo(x) for x in tqdm(abc)])

100%|██████████| 844/844 [03:37<00:00,  3.88it/s]


In [94]:
df.duplicated().sum()

20

In [95]:
df.drop_duplicates(inplace=True)

In [87]:
import pickle
with open("./kbo_player.pkl",'wb') as f:
    pickle.dump(df,f)

In [109]:
with open("./kbo_player.pkl",'rb') as f:
    df1 = pickle.load(f)
    print(df1.duplicated().value_counts())

False    824
True      20
Name: count, dtype: int64


In [142]:
df1['팀'].value_counts()

KeyError: '팀'

In [116]:
with open("./kbo_detail.pkl",'rb') as f:
    df2 = pickle.load(f)
    df2 = pd.DataFrame(df2)
    print(df2.keys())
    df2['포지션'].value_counts()

Index(['선수명', '등번호', '생년월일', '포지션', '신장/체중', '경력', '입단 계약금', '연봉', '지명순위',
       '입단년도', 'team'],
      dtype='object')


In [127]:
type(df2['포지션'].value_counts())

pandas.core.series.Series

In [131]:
df2['포지션'].value_counts().index

Index([' 투수(우투우타)', ' 내야수(우투우타)', ' 투수(좌투좌타)', ' 포수(우투우타)', ' 내야수(우투좌타)',
       ' 외야수(우투우타)', ' 외야수(우투좌타)', ' 투수(우언우타)', ' 외야수(좌투좌타)', ' 투수(우투좌타)',
       ' 포수(우투좌타)', ' 내야수(좌투좌타)', ' 외야수(우투양타)', ' 투수(좌투우타)', ' 투수(우투양타)',
       ' 내야수(우투양타)', ' 투수(우언좌타)', ' 외야수(좌투우타)', ' 포수(우투양타)'],
      dtype='object', name='포지션')

In [122]:
df2['포지션'].unique().size

19

In [146]:
df2['신장'] = df2['신장/체중'].apply(lambda x : x.split('/')[0].replace("cm","")).apply(int)
df2['체중'] = df2['신장/체중'].apply(lambda x : x.split('/')[1].replace("kg","")).apply(int)

In [147]:
df2['bmi'] = (df2['신장']/100)**2 / df2['체중']

In [148]:
df2.sort_values(by = ['bmi'], ascending=False)

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,team,신장,체중,bmi
673,강한울,No.6,1991년 09월 12일,내야수(우투좌타),181cm/66kg,사당초-중앙중-안산공고-원광대-KIA-삼성-상무,14000만원,10000만원,14 KIA 2차 1라운드 5순위,14KIA,삼성 라이온즈,181,66,0.049638
379,백선기,No.114,1998년 08월 27일,투수(좌투좌타),186cm/70kg,남도초-경상중-대구상원고-KT,5000만원,3000만원,18 KT 2차 7라운드 61순위,,LG 트윈스,186,70,0.049423
821,이태규,No.45,2000년 02월 21일,투수(우투좌타),188cm/72kg,희망대초-매향중-장안고-KIA,8000만원,3000만원,19 KIA 2차 3라운드 30순위,23KIA,KT 위즈,188,72,0.049089
804,엄상백,No.18,1996년 10월 04일,투수(우언우타),187cm/72kg,역삼초-언북중-덕수고-KT-상무,23000만원,25000만원,15 KT 1차,15KT,KT 위즈,187,72,0.048568
35,성지훈,No.97,2000년 01월 29일,투수(좌투좌타),181cm/68kg,송정동초-무등중-광주제일고-동아대,,3000만원,23 한화 육성선수,24한화,한화 이글스,181,68,0.048178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,김기연,No.45,1997년 09월 07일,포수(우투우타),178cm/106kg,광주수창초-진흥중-진흥고-LG,7000만원,4000만원,16 LG 2차 4라운드 34순위,17LG,두산 베어스,178,106,0.029891
57,이성원,No.98,1999년 11월 02일,내야수(우투우타),185cm/115kg,수원신곡초-매향중-장안고,6000만원,3000만원,18 한화 2차 6라운드 54순위,,한화 이글스,185,115,0.029761
30,박성웅,No.63,1999년 08월 20일,투수(좌투좌타),178cm/109kg,광주서석초-충장중-광주제일고,10000만원,3300만원,18 한화 2차 2라운드 14순위,18한화,한화 이글스,178,109,0.029068
356,김범석,No.55,2004년 05월 21일,포수(우투우타),178cm/110kg,김해삼성초-경남중-경남고,25000만원,3300만원,23 LG 1라운드 7순위,23LG,LG 트윈스,178,110,0.028804


In [153]:
df2.groupby(['team'],as_index=False)['bmi'].mean()

,team,bmi
0,KIA 타이거즈,0.038004
1,KT 위즈,0.039130
2,LG 트윈스,0.038731
3,NC 다이노스,0.038764
4,SSG 랜더스,0.038700
5,고양 히어로즈,0.039487
6,두산 베어스,0.038788
7,롯데 자이언츠,0.038038
8,삼성 라이온즈,0.038447
9,키움 히어로즈,0.037778


In [154]:
sum(range(1,11)) / 10

5.5

In [156]:
import numpy as np
np.array(np.arange(1,11)).var()

8.25

In [159]:
pd.DataFrame({'a':range(1,11)}).mean()

a    5.5
dtype: float64